# Example Python code for diffusion

We start with importing the print function, and the packages numpy, os, sys, and scipy's linear algebra tools. These packages will be used in the remainder of the Python script.

In [1]:
from __future__ import print_function
import numpy, os, sys
import scipy.sparse.linalg as sp_lin_alg

Then, we import our own importer for C++ code based on cython.

In [2]:
try:
  import HyperHDG
except (ImportError, ModuleNotFoundError) as error:
  sys.path.append(os.path.dirname(os.path.abspath("emaple1.ipynb")) + "/../import")
  import HyperHDG

Next, we define some basic parameters such as the polynomial degree of both, the (discontinuous) polynomials which live on the skeletal, and the polynomials which are utilized by the local solvers, the dimension of a hyperedge, the dimension of the surrounding space, the refinement

In [3]:
poly_degree = 1
hyEdge_dim  = 1
cube_dim    = 2
refinement  = 1
debug_mode  = True

Now we configure the discretization we want to work on as a textual representation, which is then fed into `cython_import`. Several aspects are combined here. First, there is the mesh topology and geometry, which in this case are hypercube Geometry::UnitCube with its topology Topology::Cubic. This geometry, like most geometries, has two dimension parameters. The first, denoted `hyEdge_dim` is the dimension of the hyper-edges, the mesh cells in conventional naming. The second is in this case `cube_dim`, which is the dimension of the hypecube. The combination here means our mesh will consist of the edges in a subdivided unit square. Thus, we are not solving a differential equation on the square, but rather on the graph consisting of the edges.

The second aspect of discretization after the mesh is the local solver of the HDG method. Local solvers are the HDG way of specifying a partial differential equation. Here, we decide for a simple diffusion problem, LocalSolvers::Diffusion. Since the HDG method is specified by local polynomials spaces, this local solver receives the polynomial degree as one of its arguments.

In [4]:
hdg_config                 = HyperHDG.config()
hdg_config.global_loop     = "Elliptic"
hdg_config.topology        = f'Cubic<{hyEdge_dim},{cube_dim}>'
hdg_config.geometry        = f'UnitCube<{hyEdge_dim},{cube_dim},double>'
hdg_config.node_descriptor = f'Cubic<{hyEdge_dim},{cube_dim}>'
hdg_config.local_solver    = "Diffusion<" + str(hyEdge_dim) + "," + str(poly_degree) + "," + str(
  2*poly_degree) + ",HG<" + str(hyEdge_dim) + ">::DiffusionElliptic,double>"
hdg_config.include_files   = ["examples/parameters/diffusion.hxx"]
hdg_config.cython_replacements = ["vector[unsigned int]", "vector[unsigned int]"]
hdg_config.debug_mode      = debug_mode

This text is compiled into C++ code and then to a linkable object file, which creates a Python class.

This in turn is used to create an object. The constructor in this case is a vector of integers of length `space_dim`, which is the constructor argument specified for Geometry::UnitCube.

In [5]:
hyperHDGClass = HyperHDG.include(hdg_config)
HDGObject   = hyperHDGClass ( [2 ** refinement] * cube_dim )

Cythonizing ... DONE without compilation in 37.13 milliseconds.


Now, we can start dealing with the actual problem. To do so, we create the global right hand side by evaluating the residual of A x - b for x = 0 and multiplying the result by -1.

In [6]:
vectorRHS = numpy.multiply( HDGObject.residual_flux(HDGObject.zero_vector()), -1. )

Python's Scipy package provides solvers for matrix-free situations. These solver require the definition of a LinearOperator, which is done in two lines:

In [7]:
system_size = HDGObject.size_of_system()
A = sp_lin_alg.LinearOperator((system_size,system_size), matvec=HDGObject.trace_to_flux)

Solving the linear system of equations can then be done using the CG method in one line. The remaining lines check that the CG method has converged.

In [8]:
[vectorSolution, num_iter] = sp_lin_alg.cg(A, vectorRHS, tol=1e-13)
if num_iter != 0:
  print("CG solver failed with a total number of ", num_iter, "iterations.")
  raise RuntimeError("Linear solvers did not converge!")

In the case that the analytical solution is known, we can evaluate the error and print it using:

In [10]:
print("Error: ", HDGObject.errors(vectorSolution)[0])

Error:  0.007850928662766413


Additionally, we can plot the calculated solution after setting some options found in PlotOptions:

In [11]:
HDGObject.plot_option( "fileName" , "diffusion_elliptic_py" )
HDGObject.plot_option( "printFileNumber" , "false" )
HDGObject.plot_option( "scale" , "0.95" )
HDGObject.plot_solution(vectorSolution)